# Import Library

In [ ]:
import os
import unicodedata

import pandas as pd
from tqdm import tqdm
import fitz  # PyMuPDF
import pickle

# Langchain 관련
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# CONFIG

In [ ]:
class CFG:
    # Root of Files
    BASE_DIRECTORY = './'
    DACON_TRAIN_CSV = './train.csv'
    DACON_TEST_CSV = './test.csv'

    EMBEDDINGS_ABOUT_TRAINING_DATASET = 'databases_multilingual-e5-base_training.pickle'
    EMBEDDINGS_ABOUT_TEST_DATASET = 'databases_multilingual-e5-base_test.pickle'

    EMBEDDING_CSV_TEST = './datas/stf_e5_base_test.csv'

    # About Embedding
    EMBEDDING_MODEL = "intfloat/multilingual-e5-base"
    CHUNK_SIZE = 800
    CHUNK_OVERLAP = 50
    
    SEARCH_TYPE = "mmr"
    SEARCH_KWARGS_K = 3
    SEARCH_KWARGS_FETCH_K = 8

### Vector DB

In [ ]:
def process_pdf(file_path, chunk_size=CFG.CHUNK_SIZE, chunk_overlap=CFG.CHUNK_OVERLAP):
    """PDF 텍스트 추출 후 chunk 단위로 나누기"""
    # PDF 파일 열기
    doc = fitz.open(file_path)
    text = ''
    # 모든 페이지의 텍스트 추출
    for page in doc:
        text += page.get_text()
    # 텍스트를 chunk로 분할
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    chunk_temp = splitter.split_text(text)
    # Document 객체 리스트 생성
    chunks = [Document(page_content=t) for t in chunk_temp]
    return chunks


def create_vector_db(chunks, model_path=CFG.EMBEDDING_MODEL):
    """FAISS DB 생성"""
    # 임베딩 모델 설정
    model_kwargs = {'device': 'cuda'}
    encode_kwargs = {'normalize_embeddings': True}
    embeddings = HuggingFaceEmbeddings(
        model_name=model_path,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    # FAISS DB 생성 및 반환
    db = FAISS.from_documents(chunks, embedding=embeddings)
    return db

def normalize_path(path):
    """경로 유니코드 정규화"""
    return unicodedata.normalize('NFC', path)


def process_pdfs_from_dataframe(df, base_directory):
    """딕셔너리에 pdf명을 키로해서 DB, retriever 저장"""
    pdf_databases = {}
    unique_paths = df['Source_path'].unique()
    
    for path in tqdm(unique_paths, desc="Processing PDFs"):
        # 경로 정규화 및 절대 경로 생성
        normalized_path = normalize_path(path)
        full_path = os.path.normpath(os.path.join(base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path
        
        pdf_title = os.path.splitext(os.path.basename(full_path))[0]
        print(f"Processing {pdf_title}...")
        
        # PDF 처리 및 벡터 DB 생성
        chunks = process_pdf(full_path)
        db = create_vector_db(chunks)
        
        # Retriever 생성
        retriever = db.as_retriever(search_type=CFG.SEARCH_TYPE, 
                                    search_kwargs={'k': 3, 'fetch_k': 8})
        
        # 결과 저장
        pdf_databases[pdf_title] = {
                'db': db,
                'retriever': retriever
        }
    return pdf_databases

### EMBEDDING 정보를 저장한 CSV 파일의 생성을 위한 FUNCTIONS

In [ ]:
def normalize_string(s):
    """유니코드 정규화"""
    return unicodedata.normalize('NFC', s)

def format_docs(docs):
    """검색된 문서들을 하나의 문자열로 포맷팅"""
    context = ""
    for doc in docs:
        context += doc.page_content
        context += '\n'
    return context

def extract_context(df, pdf_databases, data_type='train'):
    # 결과를 저장할 리스트 초기화
    results = []

    # 배치 사이즈 설정
    batch_size = 1  # 원하는 배치 크기로 설정

    # DataFrame의 각 행에 대해 처리
    for start in tqdm(range(0, len(df), batch_size), desc="Creating Q&A including RAG info"):
        # 현재 배치 선택
        batch_rows = df.iloc[start:start + batch_size]

        # 배치 내의 각 행 처리
        for _, row in batch_rows.iterrows():
            # 소스 문자열 정규화
            source = normalize_string(row['Source'])
            question = normalize_string(row['Question'])
            if data_type == 'train':
                answer = normalize_string(row['Answer'])

            # 정규화된 키로 데이터베이스 검색
            normalized_keys = {normalize_string(k): v for k, v in pdf_databases.items()}
            retriever = normalized_keys[source]['retriever']

            context = format_docs(retriever.invoke(question))

            if data_type == 'train':
                results.append({
                    "Context" : context,
                    "Question": question,
                    "Answer"  : answer,
                })
            else:
                results.append({
                    "Context" : context,
                    "Question": question,
                })

    return results


### TRAINING에 관한 CSV 파일 생성

In [ ]:
train_df = pd.read_csv(CFG.DACON_TRAIN_CSV)
with open(CFG.EMBEDDINGS_ABOUT_TRAINING_DATASET, 'rb') as f:
    train_pdf_databases = pickle.load(f)

### TEST에 관한 CSV 파일 생성

In [ ]:
test_df = pd.read_csv(CFG.DACON_TEST_CSV)
with open(CFG.EMBEDDINGS_ABOUT_TEST_DATASET, 'rb') as f:
    test_pdf_databases = pickle.load(f)

In [ ]:
test_context = extract_context(test_df, test_pdf_databases, data_type='test')

stf_test_df = pd.DataFrame(test_context)

stf_test_df.to_csv(CFG.EMBEDDING_CSV_TEST, index=False, encoding="UTF-8")